# Rod Cutting Problem

In [12]:
#prices = { i +1 : v for i, v in enumerate([1, 5, 8,9,10,17,17,20,24,30])}

In [3]:
from collections import defaultdict

In [4]:
from functools import lru_cache

In [32]:
#help(lru_cache)

In [5]:
prices = defaultdict(lambda : -float('inf'))

In [6]:
for i, v in enumerate([1, 5, 8,9,10,17,17,20,24,30]):
    prices[i+1] = v

### Cache
使用缓存

In [47]:
cache = {}
def revenue1(r):
    if r in cache:
        #print('revenue({}) is in cache'.format(r))
        return cache[r]
    r_optimal = max([prices[r]] + [(revenue(i) + revenue(r-i)) for i in range(1,r)])
    cache[r] = r_optimal
    return r_optimal

In [13]:
import time

In [75]:
start = time.time()
revenue1(20)
print(time.time()-start)

6.818771362304688e-05


### Decorate
装饰器

In [51]:
def print_hi(func):
    def _wrap(*arg, **kwargs):
        print('Function: {}'.format(func.__name__))
        return func(*arg, **kwargs)
    return _wrap

In [62]:
def add(a,b):return a+b

In [63]:
print_hi_add = print_hi(add)

In [64]:
print_hi_add(1,2)

Function: add


3

In [65]:
@print_hi
def mul(a,b):return a*b

In [66]:
mul(2,3)

Function: mul


6

### lru_cache 放在dictionary中，list不支持
使用Python自带的lru_cache,局限性：不支持list

In [91]:
@lru_cache(maxsize=2*10)
def revenue2(r):
    return max([prices[r]] + [(revenue(i) + revenue(r-i)) for i in range(1,r)])

In [92]:
start = time.time()
print(revenue2(20))
print('time:{}'.format(time.time()-start))

60
time:0.00020813941955566406


### memo
自定义memo缓存

In [4]:
from functools import wraps

In [5]:
def memo(func):
    cache = {}
    @wraps(func)
    def __wrap(*args, **kwargs):
        #print('Function:{}'.format(func.__name__))
        str_key = str(args) + str(kwargs)
        if str_key not in cache:
            result = func(*args, **kwargs)
            cache[str_key] = result
        return cache[str_key]
    return __wrap

In [11]:
@memo
def revenue3(r):
    return max([prices[r]] + [(revenue(i) + revenue(r-i)) for i in range(1,r)])

In [14]:
start = time.time()
print(revenue(8))
print('time:{}'.format(time.time()-start))

22
time:0.0002980232238769531


In [33]:
solution = {}
@memo
def revenue(r):
    split, r_star = max([(0,prices[r])] + [(i, revenue(i) + revenue(r-i)) for i in range(1,r)] , key = lambda x : x[1])
    solution[r] = (split,r-split)
    return r_star

In [36]:
revenue(18)

52

In [51]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16)}

In [52]:
def parse_solution(r, revenue_solution):
    assert r in revenue_solution
    left, right = revenue_solution[r]
    if left == 0: return [right]
    else:
        return [left] + parse_solution(right, revenue_solution)

In [53]:
parse_solution(18,solution)

[2, 6, 10]

In [60]:
def pretty_solution(splits):
    return '->'.join(map(str,splits))

In [63]:
pretty_solution(parse_solution(18,solution))

'2->6->10'

# Edit Distance

In [14]:
@memo
def get_edit_distance(string1, string2):
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    return min([get_edit_distance(string1[:-1], string2) + 1,
                    get_edit_distance(string1, string2[:-1]) + 1,
                    get_edit_distance(string1[:-1], string2[:-1]) + (0 if string1[-1] == string2[-1] else 2)])

In [15]:
get_edit_distance('shanghai','shanghi')

1

In [16]:
get_edit_distance('shanggai','shanghi')

3

In [17]:
get_edit_distance('biejing', 'beijie')

5